In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from datetime import datetime
import random
import os

def get_image_dimensions(image_path):
    """Placeholder for getting image dimensions (width, height)."""
    # Since PIL is not imported, return default values as in original code
    return 100, 100  # Adjust as needed or implement actual image dimension retrieval

def wrap_text(c, text, font_name, font_size, max_width):
    """Wrap text to fit within max_width, returning a list of lines."""
    c.setFont(font_name, font_size)
    words = str(text).split()
    lines = []
    current_line = []
    current_width = 0
    for word in words:
        word_width = c.stringWidth(word + " ", font_name, font_size)
        if current_width + word_width <= max_width:
            current_line.append(word)
            current_width += word_width
        else:
            lines.append(" ".join(current_line))
            current_line = [word]
            current_width = word_width
    if current_line:
        lines.append(" ".join(current_line))
    return lines

def format_text(value, ctx):
    """Format text with context variables, handling errors."""
    if isinstance(value, str):
        try:
            return value.format(**{k: v for k, v in ctx.items() if isinstance(v, str)})
        except (KeyError, ValueError) as e:
            print(f"Warning: Formatting failed for value '{value}': {e}")
            with open("layout_debug.log", "a") as log_file:
                log_file.write(f"[{datetime.now()}] Warning: Formatting failed for value '{value}' in {ctx['bank_name']}: {e}\n")
            return value
    return str(value)

def check_page_break(c, y_position, margin, space_needed, PAGE_HEIGHT):
    """Check if a page break is needed and reset y_position if triggered."""
    if y_position - space_needed < margin:
        c.showPage()
        y_position = PAGE_HEIGHT - margin
        print(f"Page break triggered: Needed {space_needed}pt, had {y_position - margin}pt")
        with open("layout_debug.log", "a") as log_file:
            log_file.write(f"[{datetime.now()}] Page break for {ctx['bank_name']}: Needed {space_needed}pt, had {y_position - margin}pt\n")
    return y_position

def render_logo(c, ctx, y_position, margin, PAGE_WIDTH, PAGE_HEIGHT):
    """Render the bank logo with random alignment."""
    logo_path = ctx.get('logo_path', '')
    logo_align = random.choice(['left', 'center', 'right'])
    if logo_path and os.path.exists(logo_path):
        img_width, img_height = get_image_dimensions(logo_path)
        target_width = 1.0 * inch if ctx['bank_name'].lower() == 'wells fargo' else 1.5 * inch
        aspect_ratio = img_width / img_height if img_height > 0 else 1
        target_height = target_width / aspect_ratio
        y_position = check_page_break(c, y_position, margin, target_height + 40, PAGE_HEIGHT)
        x_logo = (margin if logo_align == 'left' else 
                  PAGE_WIDTH / 2 - target_width / 2 if logo_align == 'center' else 
                  PAGE_WIDTH - margin - target_width)
        c.drawImage(logo_path, x_logo, y_position - target_height - 10, 
                    width=target_width, height=target_height, mask='auto')
        y_position -= target_height + 40
        with open("layout_debug.log", "a") as log_file:
            log_file.write(f"[{datetime.now()}] Logo rendered for {ctx['bank_name']} at y={y_position + target_height + 40}, height={target_height}, extra whitespace=40pt\n")
    else:
        print(f"Warning: Logo not rendered for {ctx['bank_name']}, invalid path: {logo_path}")
        with open("layout_debug.log", "a") as log_file:
            log_file.write(f"[{datetime.now()}] Warning: Logo not rendered for {ctx['bank_name']}, invalid path: {logo_path}\n")
        y_position = check_page_break(c, y_position, margin, 14, PAGE_HEIGHT)
        c.setFont("Helvetica", 12)
        c.drawString(margin, y_position, f"[Logo: {ctx['bank_name']}]")
        y_position -= 14
    return y_position

def render_header(c, ctx, y_position, margin, PAGE_WIDTH, PAGE_HEIGHT, usable_width):
    """Render the header with account holder and statement information."""
    y_position = check_page_break(c, y_position, margin, 28, PAGE_HEIGHT)
    c.setFont("Helvetica", 12)
    c.drawString(margin, y_position, format_text("{account_holder}", ctx))
    y_position -= 14
    address_lines = format_text("{account_holder_address}", ctx).split(',')
    for line in address_lines:
        line = line.strip()
        if line:
            y_position = check_page_break(c, y_position, margin, 14, PAGE_HEIGHT)
            c.drawString(margin, y_position, line)
            y_position -= 14
    c.setFont("Helvetica", 16)
    y_position = check_page_break(c, y_position, margin, 18, PAGE_HEIGHT)
    c.drawCentredString(PAGE_WIDTH / 2, y_position, format_text("{account_type} Statement", ctx))
    y_position -= 28
    return y_position

def render_wells_fargo_blurb(c, ctx, y_position, margin, usable_width, PAGE_HEIGHT):
    """Render the 'Your Wells Fargo' blurb for Wells Fargo statements."""
    if ctx['bank_name'].lower() != 'wells fargo':
        return y_position
    y_position = check_page_break(c, y_position, margin, 60, PAGE_HEIGHT)
    c.setFont("Helvetica-Bold", 14)
    c.drawString(margin + 8, y_position, "Your Wells Fargo")
    y_position -= 14 + 5
    c.setFont("Helvetica", 10)
    intro_text = (
        "It’s a great time to talk with a banker about how Wells Fargo’s accounts "
        "and services can help you stay competitive by saving you time and money. "
        "To find out how we can help, stop by any Wells Fargo location or call us at "
        "the number at the top of your statement."
    )
    intro_lines = wrap_text(c, intro_text, "Helvetica", 10, usable_width)
    for line in intro_lines:
        y_position = check_page_break(c, y_position, margin, 10 * 1.3, PAGE_HEIGHT)
        c.drawString(margin + 8, y_position, line)
        y_position -= 10 * 1.3
    y_position -= 20
    return y_position

def render_table(c, ctx, section, data, col_widths, x_start, y_position, margin, usable_width, PAGE_HEIGHT, font="Helvetica", font_size=10):
    """Render a table for a given section."""
    headers = section.get("headers", [])
    row_height = font_size + 4
    total_height = (len(headers) + len(data)) * row_height

    y_position = check_page_break(c, y_position, margin, total_height, PAGE_HEIGHT)
    if headers:
        c.setFont(font, font_size)
        for i, header in enumerate(headers):
            x_pos = x_start + sum(col_widths[:i])
            if i in [2, 3]:
                c.drawRightString(x_pos + col_widths[i] - 8, y_position, header)
            else:
                c.drawString(x_pos + 8, y_position, header)
        y_position -= font_size + 4

    for row in data:
        y_position = check_page_break(c, y_position, margin, row_height, PAGE_HEIGHT)
        for i, cell in enumerate(row):
            x_pos = x_start + sum(col_widths[:i])
            cell = format_text(str(cell), ctx)
            if isinstance(cell, str) and len(cell) > 50:
                cell = cell[:47] + "..."
            if section["title"] == "Account Summary" and i == 1:
                c.drawCentredString(x_pos + col_widths[i] / 2, y_position, cell)
            elif i in [2, 3]:
                c.drawRightString(x_pos + col_widths[i] - 8, y_position, cell)
            else:
                c.drawString(x_pos + 8, y_position, cell)
        y_position -= font_size + 4

    if section.get("data_key") == "transactions" and data:
        c.setStrokeColor(colors.black)
        c.line(x_start, y_position, x_start + sum(col_widths), y_position)
    return y_position

def render_pnc_summaries(c, ctx, y_position, margin, usable_width, PAGE_HEIGHT, is_two_column=False, x_start=0):
    """Render PNC's Transaction Summary and Interest Summary tables."""
    if ctx['bank_name'].lower() != 'pnc':
        return y_position

    summaries = [
        {
            "title": "Transaction Summary",
            "content": [{
                "type": "table",
                "data": [
                    ["Checks paid/written", ctx.get('summary', {}).get('checks_written', "0")],
                    ["Check-card POS transactions", ctx.get('summary', {}).get('pos_transactions', "0")],
                    ["Check-card/virtual POS PIN txn", ctx.get('summary', {}).get('pos_pin_transactions', "0")],
                    ["Total ATM transactions", ctx.get('summary', {}).get('total_atm_transactions', "0")],
                    ["PNC Bank ATM transactions", ctx.get('summary', {}).get('pnc_atm_transactions', "0")],
                    ["Other Bank ATM transactions", ctx.get('summary', {}).get('other_atm_transactions', "0")]
                ],
                "col_widths": [0.75, 0.25],
                "font": "Helvetica",
                "size": 10,
                "style": "none"
            }]
        },
        {
            "title": "Interest Summary",
            "content": [{
                "type": "table",
                "data": [
                    ["APY earned", ctx.get('summary', {}).get('apy_earned', "0.00%")],
                    ["Days in period", ctx.get('summary', {}).get('days_in_period', "30")],
                    ["Avg collected balance", ctx.get('summary', {}).get('average_collected_balance', "$0.00")],
                    ["Interest paid this period", ctx.get('summary', {}).get('interest_paid_period', "$0.00")],
                    ["YTD interest paid", ctx.get('summary', {}).get('interest_paid_ytd', "$0.00")]
                ],
                "col_widths": [0.75, 0.25],
                "font": "Helvetica",
                "size": 10,
                "style": "none"
            }]
        }
    ]

    for section in summaries:
        y_position = check_page_break(c, y_position, margin, 20, PAGE_HEIGHT)
        c.setFont("Helvetica", 13)
        c.drawString(x_start + 8, y_position, section["title"])
        y_position -= 10
        for content in section["content"]:
            data = content.get("data", [])
            col_widths = [usable_width * w for w in content.get("col_widths", [1/len(data[0])]*len(data[0]))] if not is_two_column else \
                         [(usable_width / 2 - 20) * w for w in content.get("col_widths", [1/len(data[0])]*len(data[0]))]
            y_position = render_table(c, ctx, section, data, col_widths, x_start, y_position, margin, usable_width, PAGE_HEIGHT, content["font"], content["size"])
            y_position -= 12
        y_position -= 10
    return y_position

def render_chase_daily_balance(c, ctx, y_position, margin, usable_width, PAGE_HEIGHT):
    """Render Chase's Daily Ending Balance section."""
    if ctx['bank_name'].lower() != 'chase':
        return y_position
    y_position = check_page_break(c, y_position, margin, 60, PAGE_HEIGHT)
    c.setFont("Helvetica", 14)
    c.drawString(margin + 8, y_position, "Daily Ending Balance")
    y_position -= 20
    data = [[b.get('date', ''), b.get('amount', '')] for b in ctx.get('daily_balances', [])]
    col_widths = [usable_width * w for w in [0.5, 0.5]]
    y_position = render_table(c, ctx, {"title": "Daily Ending Balance", "headers": ["Date", "Amount"]}, 
                             data, col_widths, margin, y_position, margin, usable_width, PAGE_HEIGHT)
    y_position -= 20
    return y_position

def render_section(c, ctx, section, y_position, margin, usable_width, PAGE_HEIGHT, x_start=0, col_width=0):
    """Render a single section (text or table)."""
    y_position = check_page_break(c, y_position, margin, 20, PAGE_HEIGHT)
    c.setFont("Helvetica", 14)
    if section["title"] == "Account Summary":
        col_widths = [usable_width * w for w in section["content"][0].get("col_widths", [0.75, 0.25])]
        table_width = sum(col_widths)
        box_height = (len(section["content"][0]["data"]) * (section["content"][0]["size"] + 4) + 20 + 12)
        c.setFillColor(colors.HexColor("#D3D3D3"))
        c.setStrokeColor(colors.black)
        c.rect(x_start - 8, y_position - 20 + 8 - 4.5 * (section["content"][0]["size"] + 4), table_width + 16, box_height, fill=1, stroke=1)
        c.setFillColor(colors.black)
        c.setStrokeColor(colors.black)
    c.drawString(x_start + 8, y_position, section["title"])
    y_position -= 20

    for content in section["content"]:
        font_size = content["size"]
        if content["type"] == "text":
            lines = wrap_text(c, format_text(content["value"], ctx), content["font"], font_size, col_width or usable_width)
            y_position = check_page_break(c, y_position, margin, len(lines) * (font_size + 4), PAGE_HEIGHT)
            for line in lines:
                c.setFont(content["font"], font_size)
                c.drawString(x_start + 8, y_position, line)
                y_position -= font_size + 4
        elif content["type"] == "table":
            data = content.get("data", [])
            if content.get("data_key") == "transactions":
                data = []
                for t in ctx.get("transactions", []):
                    amount = t.get("deposits_credits", "") or f"-{t.get('withdrawals_debits', '')}"
                    data.append([
                        t.get("date", ""),
                        t.get("description", ""),
                        amount,
                        t.get("ending_balance", "")
                    ])
                print(f"Transaction table data for {ctx['bank_name']}: {len(data)} rows")
                with open("layout_debug.log", "a") as log_file:
                    log_file.write(f"[{datetime.now()}] Transaction table data for {ctx['bank_name']}: {len(data)} rows\n")
            
            if not data:
                print(f"Warning: No data for table '{section['title']}' for {ctx['bank_name']}")
                with open("layout_debug.log", "a") as log_file:
                    log_file.write(f"[{datetime.now()}] Warning: No data for table '{section['title']}' for {ctx['bank_name']}\n")
                data = [["No data available"] * len(content.get("headers", []))]

            col_widths = [usable_width * w for w in content.get("col_widths", [1/len(data[0])]*len(data[0]))] if not col_width else \
                         [col_width * w for w in content.get("col_widths", [1/len(data[0])]*len(data[0]))]
            y_position = render_table(c, ctx, section, data, col_widths, x_start, y_position, margin, usable_width, PAGE_HEIGHT, content["font"], font_size)
        y_position -= 12
    return y_position

def create_dynamic_statement(ctx, output_dir="test"):
    """Create a dynamic bank statement with modular rendering."""
    # Validate ctx
    required_keys = ['bank_name', 'account_holder', 'account_holder_address', 'account_type', 
                     'summary', 'transactions', 'website', 'contact', 'customer_account_number']
    for key in required_keys:
        if key not in ctx:
            raise ValueError(f"Missing required context key: {key}")
    
    summary_keys = ['beginning_balance', 'deposits_total', 'withdrawals_total', 'ending_balance']
    for key in summary_keys:
        if key not in ctx['summary']:
            print(f"Warning: Missing summary key '{key}' for {ctx['bank_name']}, using default value")
            with open("layout_debug.log", "a") as log_file:
                log_file.write(f"[{datetime.now()}] Warning: Missing summary key '{key}' for {ctx['bank_name']}\n")
            ctx['summary'][key] = "$0.00"

    transaction_keys = ['date', 'description', 'deposits_credits', 'withdrawals_debits', 'ending_balance']
    for tx in ctx.get('transactions', []):
        for key in transaction_keys:
            if key not in tx:
                print(f"Warning: Missing transaction key '{key}' for {ctx['bank_name']}, using empty string")
                with open("layout_debug.log", "a") as log_file:
                    log_file.write(f"[{datetime.now()}] Warning: Missing transaction key '{key}' for {ctx['bank_name']}\n")
                tx[key] = ""

    bank_name = ctx['bank_name']
    output_file = os.path.join(output_dir, f"{bank_name.lower()}_statement_{ctx['customer_account_number'][-4:]}.pdf")
    c = canvas.Canvas(output_file, pagesize=letter)
    PAGE_WIDTH, PAGE_HEIGHT = letter
    margin = 0.5 * inch
    usable_width = PAGE_WIDTH - 2 * margin
    y_position = PAGE_HEIGHT - margin
    MIN_SPACE = 80

    # Render logo and header
    y_position = render_logo(c, ctx, y_position, margin, PAGE_WIDTH, PAGE_HEIGHT)
    y_position = render_header(c, ctx, y_position, margin, PAGE_WIDTH, PAGE_HEIGHT, usable_width)

    # Define middle sections
    middle_sections = [
        {
            "title": "Important Account Information",
            "content": [{
                "type": "text",
                "value": (
                    "Effective July 1, 2025, the monthly service fee for {account_type} accounts will increase to $15 unless you maintain a minimum daily balance of $1,500, have $500 in qualifying direct deposits, or maintain a linked savings account with a balance of $5,000 or more. "
                    "For questions, visit {website} or call {contact}."
                ),
                "font": "Helvetica",
                "size": 10,
                "wrap": True
            }]
        },
        {
            "title": "Account Summary",
            "content": [{
                "type": "table",
                "data": [
                    ["Beginning Balance", ctx['summary']['beginning_balance']],
                    ["Deposits and Credits", ctx['summary']['deposits_total']],
                    ["Withdrawals and Debits", ctx['summary']['withdrawals_total']],
                    ["Ending Balance", ctx['summary']['ending_balance']]
                ],
                "col_widths": [0.75, 0.25],
                "font": "Helvetica",
                "size": 10,
                "style": "none"
            }]
        },
        {
            "title": "Transaction History",
            "content": [{
                "type": "table",
                "data_key": "transactions",
                "headers": ["Date", "Description", "Amount", "Balance"],
                "col_widths": [0.15, 0.45, 0.20, 0.20],
                "font": "Helvetica",
                "size": 10,
                "style": "none"
            }]
        }
    ]

    # Render Wells Fargo blurb before Important Account Information
    y_position = render_wells_fargo_blurb(c, ctx, y_position, margin, usable_width, PAGE_HEIGHT)

    # Coinflip for layout
    coinflip = random.randint(0, 1)
    print(f"Coinflip for {bank_name}: {coinflip} ({'sequential' if coinflip == 0 else 'two-column'})")
    with open("layout_debug.log", "a") as log_file:
        log_file.write(f"[{datetime.now()}] Coinflip for {bank_name}: {coinflip} ({'sequential' if coinflip == 0 else 'two-column'})\n")

    if coinflip == 0:
        # Sequential layout
        # Render PNC summaries before Transaction History
        if bank_name.lower() == 'pnc':
            y_position = render_pnc_summaries(c, ctx, y_position, margin, usable_width, PAGE_HEIGHT)

        for section in middle_sections:
            y_position = render_section(c, ctx, section, y_position, margin, usable_width, PAGE_HEIGHT)
    else:
        # Two-column layout
        col_width = usable_width / 2 - 10
        max_y_position = y_position
        section_titles = [section["title"] for section in middle_sections[:2]]
        align_sections = "Account Summary" in section_titles and "Important Account Information" in section_titles
        offset_y = 0
        if not align_sections:
            longest_section_idx = 0 if len(middle_sections[0]["content"][0].get("data", [])) >= len(middle_sections[1]["content"][0].get("data", [])) else 1
            offset_y = (len(middle_sections[1]["content"][0].get("data", [])) - len(middle_sections[0]["content"][0].get("data", []))) * 14 / 2

        for i, section in enumerate(middle_sections[:2]):
            x_col = margin if i == 0 else margin + col_width + 20
            y_col = max_y_position + (offset_y if (i == 1 and not align_sections) else 0)
            y_col = render_section(c, ctx, section, y_col, margin, usable_width, PAGE_HEIGHT, x_col, col_width)
            y_position = min(y_position, y_col)

        # Render PNC summaries side by side with Transaction History if present
        if bank_name.lower() == 'pnc' and middle_sections[2]["title"] == "Transaction History":
            x_col = margin
            y_col = y_position
            y_col = render_pnc_summaries(c, ctx, y_col, margin, usable_width, PAGE_HEIGHT, is_two_column=True, x_start=x_col)
            x_col = margin + col_width + 20
            y_col = render_section(c, ctx, middle_sections[2], y_col, margin, usable_width, PAGE_HEIGHT, x_col, col_width)
            y_position = min(y_position, y_col)
        else:
            y_position = render_section(c, ctx, middle_sections[2], y_position, margin, usable_width, PAGE_HEIGHT)

    # Render Chase Daily Ending Balance before footer
    y_position = render_chase_daily_balance(c, ctx, y_position, margin, usable_width, PAGE_HEIGHT)

    # Render Footer
    y_position = check_page_break(c, y_position, margin, 60, PAGE_HEIGHT)
    c.setStrokeColor(colors.black)
    c.line(margin, y_position + 20, margin + usable_width, y_position + 20)
    c.setFont("Helvetica", 8)
    lines = wrap_text(c,
        format_text(
            "All account transactions are subject to the {bank_name} Deposit Account Agreement, available at {website}. "
            "Interest rates and Annual Percentage Yields (APYs) may change without notice. "
            "For details on overdraft policies and fees, visit {website}/overdraft or call {contact}. "
            f"© 2025 {bank_name} Bank, N.A. All rights reserved. Member FDIC.",
            ctx
        ),
        "Helvetica", 8, usable_width
    )
    for line in lines:
        y_position = check_page_break(c, y_position, margin, 10, PAGE_HEIGHT)
        c.drawString(margin + 8, y_position, line)
        y_position -= 10
    y_position -= 12

    c.save()
    print(f"PDF generated: {output_file}")
    with open("layout_debug.log", "a") as log_file:
        log_file.write(f"[{datetime.now()}] PDF generated: {output_file}\n")

if __name__ == "__main__":
    with open("layout_debug.log", "a") as log_file:
        log_file.write(f"\n[{datetime.now()}] Run started\n")
    for ctx in [citi_ctx, chase_ctx, wellsfargo_ctx, pnc_ctx]:
        create_dynamic_statement(ctx)